In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Standarder Challenge').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [3]:
train = spark.read.csv('Santander Value Prediction Challenge/train.csv', inferSchema=True, header=True)

In [4]:
test = spark.read.csv('Santander Value Prediction Challenge/test.csv', inferSchema=True, header=True)

In [7]:
train.printSchema()

root
 |-- ID: string (nullable = true)
 |-- target: double (nullable = true)
 |-- 48df886f9: double (nullable = true)
 |-- 0deb4b6a8: integer (nullable = true)
 |-- 34b15f335: double (nullable = true)
 |-- a8cb14b00: integer (nullable = true)
 |-- 2f0771a37: integer (nullable = true)
 |-- 30347e683: integer (nullable = true)
 |-- d08d1fbe3: integer (nullable = true)
 |-- 6ee66e115: integer (nullable = true)
 |-- 20aa07010: double (nullable = true)
 |-- dc5a8f1d8: double (nullable = true)
 |-- 11d86fa6a: double (nullable = true)
 |-- 77c9823f2: integer (nullable = true)
 |-- 8d6c2a0b2: integer (nullable = true)
 |-- 4681de4fd: integer (nullable = true)
 |-- adf119b9a: integer (nullable = true)
 |-- cff75dd09: double (nullable = true)
 |-- 96f83a237: integer (nullable = true)
 |-- b8a716ebf: double (nullable = true)
 |-- 6c7a4567c: integer (nullable = true)
 |-- 4fcfd2b4d: integer (nullable = true)
 |-- f3b9c0b95: double (nullable = true)
 |-- 71cebf11c: integer (nullable = true)
 |-- d9

In [124]:
test.printSchema()

root
 |-- ID: string (nullable = true)
 |-- 48df886f9: double (nullable = true)
 |-- 0deb4b6a8: double (nullable = true)
 |-- 34b15f335: double (nullable = true)
 |-- a8cb14b00: double (nullable = true)
 |-- 2f0771a37: double (nullable = true)
 |-- 30347e683: double (nullable = true)
 |-- d08d1fbe3: double (nullable = true)
 |-- 6ee66e115: double (nullable = true)
 |-- 20aa07010: double (nullable = true)
 |-- dc5a8f1d8: double (nullable = true)
 |-- 11d86fa6a: double (nullable = true)
 |-- 77c9823f2: double (nullable = true)
 |-- 8d6c2a0b2: double (nullable = true)
 |-- 4681de4fd: double (nullable = true)
 |-- adf119b9a: double (nullable = true)
 |-- cff75dd09: double (nullable = true)
 |-- 96f83a237: double (nullable = true)
 |-- b8a716ebf: double (nullable = true)
 |-- 6c7a4567c: double (nullable = true)
 |-- 4fcfd2b4d: double (nullable = true)
 |-- f3b9c0b95: double (nullable = true)
 |-- 71cebf11c: double (nullable = true)
 |-- d966ac62c: double (nullable = true)
 |-- 68b647452: do

In [25]:
from pyspark.sql.functions import lit

In [57]:
test = test.withColumn("target", lit(0))

In [58]:
test.printSchema()

root
 |-- ID: string (nullable = true)
 |-- 48df886f9: double (nullable = true)
 |-- 0deb4b6a8: double (nullable = true)
 |-- 34b15f335: double (nullable = true)
 |-- a8cb14b00: double (nullable = true)
 |-- 2f0771a37: double (nullable = true)
 |-- 30347e683: double (nullable = true)
 |-- d08d1fbe3: double (nullable = true)
 |-- 6ee66e115: double (nullable = true)
 |-- 20aa07010: double (nullable = true)
 |-- dc5a8f1d8: double (nullable = true)
 |-- 11d86fa6a: double (nullable = true)
 |-- 77c9823f2: double (nullable = true)
 |-- 8d6c2a0b2: double (nullable = true)
 |-- 4681de4fd: double (nullable = true)
 |-- adf119b9a: double (nullable = true)
 |-- cff75dd09: double (nullable = true)
 |-- 96f83a237: double (nullable = true)
 |-- b8a716ebf: double (nullable = true)
 |-- 6c7a4567c: double (nullable = true)
 |-- 4fcfd2b4d: double (nullable = true)
 |-- f3b9c0b95: double (nullable = true)
 |-- 71cebf11c: double (nullable = true)
 |-- d966ac62c: double (nullable = true)
 |-- 68b647452: do

In [59]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [60]:
train_columns = train.columns
train_features = train_columns[2:]
test_columns = test.columns
test_features = test_columns[1:]
test_features = test_columns[:4990]



In [61]:
test_features

['ID',
 '48df886f9',
 '0deb4b6a8',
 '34b15f335',
 'a8cb14b00',
 '2f0771a37',
 '30347e683',
 'd08d1fbe3',
 '6ee66e115',
 '20aa07010',
 'dc5a8f1d8',
 '11d86fa6a',
 '77c9823f2',
 '8d6c2a0b2',
 '4681de4fd',
 'adf119b9a',
 'cff75dd09',
 '96f83a237',
 'b8a716ebf',
 '6c7a4567c',
 '4fcfd2b4d',
 'f3b9c0b95',
 '71cebf11c',
 'd966ac62c',
 '68b647452',
 'c88d108c9',
 'ff7b471cd',
 'd5308d8bc',
 '0d866c3d7',
 'bc3f77679',
 'bd8f989f1',
 '0eff5bf95',
 '22ed6dba3',
 '92b13ebba',
 'c330f1a67',
 '233c7c17c',
 '2cb4d123e',
 'eeac16933',
 '87ffda550',
 '822e49b95',
 '316b978cd',
 'd04e16aed',
 '5d5c5ce6d',
 'ec863cb52',
 '11ad148bd',
 'ea18d720e',
 '408d86ce9',
 '69d8b4020',
 'b0868a049',
 '80b14398e',
 'ecdef52b2',
 'c36bcacc5',
 '151d318cd',
 'ab0cddb90',
 '2d6bd8275',
 'dfdf4b580',
 'c87722678',
 'a8c320153',
 '7df8788e8',
 'a04684f1f',
 'fa977f17b',
 'feed9d437',
 '645b47cde',
 '7298ca1ef',
 'd80abf8bc',
 '654dd8a3b',
 '7fc39583c',
 '8c94b6675',
 'e421c414e',
 '5b91580ee',
 '964335fdf',
 '5a86cabd0',

In [62]:
assemlber = VectorAssembler(inputCols=train_features, outputCol='features')
assemlber2 = VectorAssembler(inputCols=test_features, outputCol='features')

In [63]:
output = assemlber.transform(train)
output2 = assemlber.transform(test)

In [66]:
output.printSchema()

root
 |-- ID: string (nullable = true)
 |-- target: double (nullable = true)
 |-- 48df886f9: double (nullable = true)
 |-- 0deb4b6a8: integer (nullable = true)
 |-- 34b15f335: double (nullable = true)
 |-- a8cb14b00: integer (nullable = true)
 |-- 2f0771a37: integer (nullable = true)
 |-- 30347e683: integer (nullable = true)
 |-- d08d1fbe3: integer (nullable = true)
 |-- 6ee66e115: integer (nullable = true)
 |-- 20aa07010: double (nullable = true)
 |-- dc5a8f1d8: double (nullable = true)
 |-- 11d86fa6a: double (nullable = true)
 |-- 77c9823f2: integer (nullable = true)
 |-- 8d6c2a0b2: integer (nullable = true)
 |-- 4681de4fd: integer (nullable = true)
 |-- adf119b9a: integer (nullable = true)
 |-- cff75dd09: double (nullable = true)
 |-- 96f83a237: integer (nullable = true)
 |-- b8a716ebf: double (nullable = true)
 |-- 6c7a4567c: integer (nullable = true)
 |-- 4fcfd2b4d: integer (nullable = true)
 |-- f3b9c0b95: double (nullable = true)
 |-- 71cebf11c: integer (nullable = true)
 |-- d9

In [67]:
output2.printSchema()

root
 |-- ID: string (nullable = true)
 |-- 48df886f9: double (nullable = true)
 |-- 0deb4b6a8: double (nullable = true)
 |-- 34b15f335: double (nullable = true)
 |-- a8cb14b00: double (nullable = true)
 |-- 2f0771a37: double (nullable = true)
 |-- 30347e683: double (nullable = true)
 |-- d08d1fbe3: double (nullable = true)
 |-- 6ee66e115: double (nullable = true)
 |-- 20aa07010: double (nullable = true)
 |-- dc5a8f1d8: double (nullable = true)
 |-- 11d86fa6a: double (nullable = true)
 |-- 77c9823f2: double (nullable = true)
 |-- 8d6c2a0b2: double (nullable = true)
 |-- 4681de4fd: double (nullable = true)
 |-- adf119b9a: double (nullable = true)
 |-- cff75dd09: double (nullable = true)
 |-- 96f83a237: double (nullable = true)
 |-- b8a716ebf: double (nullable = true)
 |-- 6c7a4567c: double (nullable = true)
 |-- 4fcfd2b4d: double (nullable = true)
 |-- f3b9c0b95: double (nullable = true)
 |-- 71cebf11c: double (nullable = true)
 |-- d966ac62c: double (nullable = true)
 |-- 68b647452: do

In [71]:
final_train = output.select('target','features')
final_test = output2.select('target','features')

In [72]:
final_train.head(1)

[Row(target=38000000.0, features=SparseVector(4991, {37: 1300000.0, 108: 4333333.34, 118: 13200000.0, 215: 6000000.0, 291: 7100000.0, 458: 1600000.0, 498: 6600000.0, 511: 2000000.0, 562: 3205000.0, 572: 400000.0, 678: 400000.0, 713: 8300000.0, 724: 12800000.0, 746: 40000000.0, 878: 900000.0, 880: 6200000.0, 916: 1600000.0, 1057: 600000.0, 1086: 7200000.0, 1104: 2000000.0, 1127: 40000000.0, 1210: 400000.0, 1303: 3000000.0, 1325: 2000000.0, 1330: 6000000.0, 1357: 8000000.0, 1391: 24400000.0, 1447: 2000000.0, 1452: 1500000.0, 1524: 500000.0, 1580: 1000000.0, 1594: 20000000.0, 1602: 4000000.0, 1666: 1000000.0, 1717: 400000.0, 1775: 2600000.0, 1803: 600000.0, 1865: 28000000.0, 1995: 700000.0, 2101: 4000000.0, 2133: 1600000.0, 2199: 2800000.0, 2212: 3600000.0, 2231: 2000000.0, 2243: 950000.0, 2318: 400000.0, 2325: 2000000.0, 2355: 800000.0, 2394: 6000000.0, 2500: 12066666.66, 2582: 500000.0, 2630: 2200000.0, 2755: 600000.0, 2788: 800000.0, 2801: 9300000.0, 2848: 6000000.0, 2875: 400000.0, 30

In [73]:
final_test.head(1)

[Row(target=0, features=SparseVector(4991, {26: 5528249.3098, 50: 8175879.759, 79: 281465.9857, 170: 1370.851, 279: 9666604.4042, 281: 215443.6142, 335: 3777239.2062, 391: 559468.5841, 683: 5844694.0883, 693: 106923.3493, 793: 7946521.7953, 864: 3682277.7086, 929: 5341349.7503, 954: 834202.5312, 974: 21318494.0882, 1007: 2567881.0539, 1020: 53879977.4695, 1212: 2068647.6678, 1248: 25948350.6186, 1268: 25368649.9183, 1317: 30224.4601, 1327: 1254864.4474, 1455: 10814.0834, 1501: 71655.7416, 1694: 427077.7016, 1786: 375358.6892, 1908: 104947760.035, 1927: 5992036.5719, 1963: 12008.024, 1992: 10163658.5527, 2117: 9025470.1394, 2134: 2374450.1487, 2208: 6877070.5117, 2264: 21507.2332, 2312: 1373878.7337, 2319: 2073588.2892, 2340: 2798985.492, 2394: 202633870.828, 2410: 15429828.9183, 2455: 42352220.8013, 2525: 6186726.0256, 2559: 305324.1074, 2635: 1088803.8057, 2694: 3549.2276, 2769: 318249.1014, 3083: 1166.8405, 3091: 155588988.812, 3098: 1084997.2815, 3106: 29906787.5716, 3206: 7747827.6

In [74]:
final_train.describe().show()

+-------+-----------------+
|summary|           target|
+-------+-----------------+
|  count|             4459|
|   mean|5944923.322036331|
| stddev|8234311.501629657|
|    min|          30000.0|
|    max|            4.0E7|
+-------+-----------------+



In [75]:
final_test.describe().show()

+-------+------+
|summary|target|
+-------+------+
|  count| 49342|
|   mean|   0.0|
| stddev|   0.0|
|    min|     0|
|    max|     0|
+-------+------+



In [76]:
lr = LinearRegression(featuresCol='features',labelCol='target',predictionCol='prediction')

In [77]:
lr_model = lr.fit(final_train)

In [78]:
test_results = lr_model.evaluate(final_test)

In [79]:
prediction_data = lr_model.transform(final_test).show()

+------+--------------------+--------------------+
|target|            features|          prediction|
+------+--------------------+--------------------+
|     0|(4991,[26,50,79,1...|-3.31761711334393...|
|     0|(4991,[121,248,47...|1.0903456706728373E8|
|     0|(4991,[41,107,108...|   9286921.929719482|
|     0|(4991,[29,35,37,1...|   8418269.839964252|
|     0|(4991,[1950,2228,...| 7.257170804916306E7|
|     0|(4991,[207,369,43...|2.1102768370498756E8|
|     0|(4991,[224,405,43...| 7.453430093868433E7|
|     0|(4991,[291,3232,3...|   1499617.946925191|
|     0|(4991,[2,17,99,57...|-1.19625202770994...|
|     0|(4991,[8,37,38,83...|   9976518.287066046|
|     0|(4991,[8,13,31,20...|1.9591667949846387E7|
|     0|(4991,[2455,3175,...|-1.522895713124073E7|
|     0|(4991,[182,559,60...| 3.450537904496205E7|
|     0|(4991,[315,850,11...|   -3897377.07734843|
|     0|(4991,[10,188,193...| 3.360208009285807E7|
|     0|(4991,[8,14,23,54...|5.1160237505053066E7|
|     0|(4991,[291,296,30...|-3

In [80]:
test_results.rootMeanSquaredError

19583603584.753563